In [ ]:
!pip install keras_tuner

In [85]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras_tuner
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

## Scale down values to (0,1)

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters = hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
            kernel_size = hp.Choice('conv_1_kernel',values = [3,5]),
            activation = 'relu', input_shape = (28,28,1)
        ),
        keras.layers.Conv2D(
            filters = hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
            kernel_size = hp.Choice('conv_2_kernel',values = [3,5]),
            activation = 'relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units = hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
            activation = 'relu'
        ),
        keras.layers.Dense(10,activation='softmax')
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                  loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
    return model

In [10]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name = 'Mnist Fashion')

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 53s]
val_accuracy: 0.8799999952316284

Best val_accuracy So Far: 0.918666660785675
Total elapsed time: 00h 04m 26s
INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        13856     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1179712   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,194,698
Trainable params: 1,194,698
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1475 - accuracy: 0.9449 - val_loss: 0.2755 - val_accuracy: 0.9095
Epoch 5/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.1094 - accuracy: 0.9599 - val_loss: 0.2893 - val_accuracy: 0.9130
Epoch 6/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0790 - accuracy: 0.9711 - val_loss: 0.3067 - val_accuracy: 0.9140
Epoch 7/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0553 - accuracy: 0.9803 - val_loss: 0.3746 - val_accuracy: 0.9118
Epoch 8/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0441 - accuracy: 0.9834 - val_loss: 0.4133 - val_accuracy: 0.9108
Epoch 9/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0351 - accuracy: 0.9867 - val_loss: 0.4810 - val_accuracy: 0.9088
Epoch 10/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0269 - accuracy: 0.9898 - val_loss: 0.5226 - val_a

In [16]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model/assets


In [17]:
from keras.models import  load_model
model = load_model('my_model')

In [92]:
#img = tf.keras.preprocessing.image.array_to_img(test_images[0])
pred_output = model.predict(test_images)
pred_labels = np.argmax(pred_output, axis=1)
model.evaluate(test_images,test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.5364 - accuracy: 0.9088


[0.5363530516624451, 0.9088000059127808]

In [96]:
confusion_matrix(test_labels,pred_labels,labels=None)

array([[866,   4,  13,  10,   3,   1,  96,   0,   7,   0],
       [  1, 975,   0,  16,   2,   0,   4,   0,   2,   0],
       [ 19,   2, 854,  11,  44,   0,  67,   0,   3,   0],
       [ 27,   2,  11, 918,  18,   0,  21,   0,   3,   0],
       [  2,   1,  78,  25, 841,   0,  51,   0,   2,   0],
       [  0,   0,   0,   0,   0, 971,   0,  21,   1,   7],
       [130,   1,  40,  24,  54,   0, 737,   0,  14,   0],
       [  0,   0,   0,   0,   0,   4,   0, 975,   0,  21],
       [  3,   0,   1,   4,   2,   1,   5,   3, 980,   1],
       [  0,   0,   0,   0,   0,   5,   0,  24,   0, 971]])

In [97]:
classification_report(test_labels,pred_labels,labels=None)

'              precision    recall  f1-score   support\n\n           0       0.83      0.87      0.85      1000\n           1       0.99      0.97      0.98      1000\n           2       0.86      0.85      0.86      1000\n           3       0.91      0.92      0.91      1000\n           4       0.87      0.84      0.86      1000\n           5       0.99      0.97      0.98      1000\n           6       0.75      0.74      0.74      1000\n           7       0.95      0.97      0.96      1000\n           8       0.97      0.98      0.97      1000\n           9       0.97      0.97      0.97      1000\n\n    accuracy                           0.91     10000\n   macro avg       0.91      0.91      0.91     10000\nweighted avg       0.91      0.91      0.91     10000\n'